# [IAPR 2019:][iapr2019] Lab 2 ‒  Object description

**Author:** Adrien Lüthi, Elias Gajo  
**Due date:** 19.04.2019

[iapr2019]: https://github.com/LTS5/iapr-2019

## Extract relevant data
We first need to extract the `lab-02-data.tar.gz` archive.
To this end, we use the [tarfile] module from the Python standard library.

[tarfile]: https://docs.python.org/3.6/library/tarfile.html

In [1]:
import tarfile
import os

data_base_path = os.path.join(os.pardir, 'data')
data_folder = 'lab-02-data'
tar_path = os.path.join(data_base_path, data_folder + '.tar.gz')
with tarfile.open(tar_path, mode='r:gz') as tar:
    tar.extractall(path=data_base_path)

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\lab-02-data.tar.gz'

## Description
In the `lab-02-data/` folder, you will find 28x28 grey-scale pictures of handwritten "0" and "1".
These digits have been extracted from MNIST dataset (http://yann.lecun.com/exdb/mnist/).

Your goal is to extract, from each of those images, a 2-dimensional feature vector (i.e. 2 features) and to plot them all on a 2D graph.
If you have chosen good features, the vectors of the "0"'s should nicely cluster in one part of the plane and those of the "1"'s in another.

Please try first the Fourier Descriptors.
You can make several attempts: e.g. with and without invariance to rotation, translation, scaling, etc.
You can also for instance rotate the images and assess the invariance in rotation.

**Note:** for the Fourier descriptors, the u_k signal has to be constructed by following the contour point after point.
Some pre-processing (image binarization, possibly some Mathematical Morphology) might be useful.

Then feel free to try other features, the more you try, the better it will be (for you).

### 1.1 Data visualization

In [ ]:
import skimage.io
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Load images
data_base_path = os.path.join(os.pardir, 'data')
data_folder = 'lab-02-data'
#  Load zeros
zeros_path = os.path.join(data_base_path, data_folder, '0')
zeros_names = [nm for nm in os.listdir(zeros_path) if '.png' in nm]  # make sure to only load .png
zeros_names.sort()  # sort file names
ic = skimage.io.imread_collection([os.path.join(zeros_path, nm) for nm in zeros_names])
zeros_im = skimage.io.concatenate_images(ic)
#  Load ones
ones_path = os.path.join(data_base_path, data_folder, '1')
ones_names = [nm for nm in os.listdir(ones_path) if '.png' in nm]  # make sure to only load .png
ones_names.sort()  # sort file names
ic = skimage.io.imread_collection(([os.path.join(ones_path, nm) for nm in ones_names]))
ones_im = skimage.io.concatenate_images(ic)

# Plot images
fig, axes = plt.subplots(2, len(zeros_im), figsize=(12, 3))
for ax, im, nm in zip(axes[0], zeros_im, zeros_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)
for ax, im, nm in zip(axes[1], ones_im, ones_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)

### 1.2 Fourier descriptors
Add your implementation and discussion

Nous allons tester ici de considérer nos features comme les composantes de la DFT du contour de chaque image. Comme nous voulons que notre code soit robuste, il faut s'assurer que l'algorithme soit invariant par translation, rotation et par changement d'échelle. Pour nous assurer d'être invariant par rotation, nous considérons uniquement la magnitude de la DFT. Pour être invariant par translation, nous ne considérons pas la composante DC de la DFT (fréquence nulle). Et enfin, pour être invariant par changement d'échelle, nos features sont le rapport entre f2 et f1, et le rapport entre f3 et f1. 

In [ ]:
def img_binarize(image, th):
    
    image_bin = image.copy()
    
    for i in range(len(image_bin)):
        for j in range(len(image_bin)):
            #print(zeros_im[img][i,j])
            if image_bin[i,j] > th: 
                image_bin[i,j] = 1

            else: image_bin[i,j] = 0
                       
    return image_bin


def find(list1, val, index=None): 
    indices = []
    # traverse in the list 
    if index == None: 
        for i in range(len(list1)): 
            # compare with all the values 
            # with val 
            if list1[i] == val: 
                indices.append(i) 
        return indices
    
    else: 
        for i in range(len(index)):
            if list1[index[i]] == val: 
                return index[i]
            
def fullHoles(img_collection):
    img_collection_copy = np.copy(img_collection)
    nb_of_picture, h, w = img_collection_copy.shape
    for im in img_collection_copy:
        # remplissage en détectant le premier et dernier pixel de chaque colonne
        column = 0
        while (column < w):

            pixel_of_column = 0;
            #search the first and last white pixels
            first = 0;
            while(first < h):
                if(im[first, column] > 0):
                    break
                first = first + 1

            last = (h-1);
            while(last > 0):
                if(im[last, column] > 0):
                    break
                last = last - 1

            if((first == (h-1)) or (last == 0)):
                column = column + 1
                continue
            else:
                for i in range(first, last):
                    im[i,column] = 255
            column = column + 1
        
    return img_collection_copy
      

In [ ]:
import skimage.io
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cmath
from skimage.morphology import square
%matplotlib inline

zeros_im_bin = zeros_im.copy()
ones_im_bin = ones_im.copy()
    
zeros_im_edge = []
ones_im_edge = []

zeros_im_edge_bin = []
ones_im_edge_bin = []

zeros_list = dict()
ones_list = dict() 

zeros_x_sort = dict()
zeros_y_sort = dict()

ones_x_sort = dict()
ones_y_sort = dict()

u_k_zeros = dict()
u_k_ones = dict()

f_l_zeros = dict()
f_l_ones = dict()

contour_zeros_pixel_nbr = np.zeros(len(zeros_im_bin))
contour_ones_pixel_nbr = np.zeros(len(zeros_im_bin))

featureZeros = np.zeros((len(zeros_im_bin), 2))
featureOnes = np.zeros((len(zeros_im_bin), 2))


#Binarize the images
for img in range(len(zeros_im_bin)):
    zeros_im_bin[img]= img_binarize(zeros_im[img], 70)
    ones_im_bin[img] = img_binarize(ones_im[img], 150)   


for img in range(len(zeros_im_bin)):

    zeros_list[img] = []
    ones_list[img] = []

    zeros_x_sort[img] = []
    zeros_y_sort[img] = []
    ones_x_sort[img] = []
    ones_y_sort[img] = []
    
    u_k_zeros[img] = []
    u_k_ones[img] = []
    f_l_zeros[img] = []
    f_l_ones[img] = []

    st_point_zeros = np.zeros(2).astype(int)
    st_point_ones = np.zeros(2).astype(int)
    
    #Fill the hole
    zeros_full = fullHoles(zeros_im_bin)
    ones_full = fullHoles(ones_im_bin)
    
    #Find the contours
    zeros_im_edge.append(skimage.filters.roberts(zeros_full[img], mask=None))
    ones_im_edge.append(skimage.filters.roberts(ones_full[img], mask=None))  
    
    #Binarize the contour images
    zeros_im_edge_bin.append(img_binarize(zeros_im_edge[img], 0))
    ones_im_edge_bin.append(img_binarize(ones_im_edge[img], 0))
        
    
    ones_list[0].append(np.where(ones_im_edge_bin[0]))

    #gets coordinates of white pixels
    zeros_list[img].append(np.where(zeros_im_edge_bin[img]))
    ones_list[img].append(np.where(ones_im_edge_bin[img]))
        
    #compute the number of pixel in each contour
    contour_zeros_pixel_nbr[img] = len(zeros_list[img][0][1])
    contour_ones_pixel_nbr[img] = len(ones_list[img][0][1])
        
    #Find the coordinate of the first white pixel
    x_current_zero = zeros_list[img][0][1][0]
    y_current_zero = zeros_list[img][0][0][0]
    zeros_x_sort[img].append(x_current_zero)
    zeros_y_sort[img].append(y_current_zero)
    zeros_list[img][0][1][0] = -1
    zeros_list[img][0][0][0] = -1
    
    x_current_one = ones_list[img][0][1][0]
    y_current_one = ones_list[img][0][0][0]
    ones_x_sort[img].append(x_current_one)
    ones_y_sort[img].append(y_current_one)
    ones_list[img][0][1][0] = -1
    ones_list[img][0][0][0] = -1
    
    #print("je traite le 0 = ", img)
    while len(zeros_x_sort[img]) != len(zeros_list[img][0][1]):
        
        #up
        mask_up = np.isin(zeros_list[img][0][1], x_current_zero)
        indices_up = find(mask_up, True)

        #right
        mask_right = np.isin(zeros_list[img][0][0], y_current_zero)
        indices_right = find(mask_right, True)

        #down
        mask_down = np.isin(zeros_list[img][0][1], x_current_zero)
        indices_down = find(mask_up, True)

        #left
        mask_right = np.isin(zeros_list[img][0][0], y_current_zero)
        indices_left = find(mask_right, True)

        #up
        if find(zeros_list[img][0][0], y_current_zero - 1, indices_up) != None:
            zeros_list[img][0][0][find(zeros_list[img][0][0], y_current_zero - 1, indices_up)] = -1
            y_current_zero = y_current_zero - 1
            zeros_x_sort[img].append(x_current_zero)
            zeros_y_sort[img].append(y_current_zero)
            #print("up")
            
        #right
        elif find(zeros_list[img][0][1], x_current_zero + 1, indices_right) != None:
            zeros_list[img][0][1][find(zeros_list[img][0][1], x_current_zero + 1, indices_right)] = -1
            x_current_zero = x_current_zero + 1
            zeros_x_sort[img].append(x_current_zero)
            zeros_y_sort[img].append(y_current_zero)
            #print("right")
            
        #down      
        elif find(zeros_list[img][0][0], y_current_zero + 1, indices_down) != None:
            zeros_list[img][0][0][find(zeros_list[img][0][0], y_current_zero + 1, indices_down)] = -1
            y_current_zero = y_current_zero + 1
            zeros_x_sort[img].append(x_current_zero)
            zeros_y_sort[img].append(y_current_zero) 
            #print("down")
            
        #left   
        elif find(zeros_list[img][0][1], x_current_zero - 1, indices_left) != None:
            zeros_list[img][0][1][find(zeros_list[img][0][1], x_current_zero - 1, indices_left)] = -1
            x_current_zero = x_current_zero - 1
            zeros_x_sort[img].append(x_current_zero)
            zeros_y_sort[img].append(y_current_zero)
            #print("left")
            
        else: 
            break
      
    
    #Sort contour for ones
    while len(ones_x_sort[img]) != len(ones_list[img][0][1]):
        
        #up
        mask_up = np.isin(ones_list[img][0][1], x_current_one)
        indices_up = find(mask_up, True)

        #right
        mask_right = np.isin(ones_list[img][0][0], y_current_one)
        indices_right = find(mask_right, True)

        #down
        mask_down = np.isin(ones_list[img][0][1], x_current_one)
        indices_down = find(mask_up, True)

        #left
        mask_right = np.isin(ones_list[img][0][0], y_current_one)
        indices_left = find(mask_right, True)
        
        #up
        if find(ones_list[img][0][0], y_current_one - 1, indices_up) != None:
            ones_list[img][0][0][find(ones_list[img][0][0], y_current_one - 1, indices_up)] = -1
            y_current_one = y_current_one - 1
            ones_x_sort[img].append(x_current_one)
            ones_y_sort[img].append(y_current_one)
            #print("up")
            
        #right
        elif find(ones_list[img][0][1], x_current_one + 1, indices_right) != None:
            ones_list[img][0][1][find(ones_list[img][0][1], x_current_one + 1, indices_right)] = -1
            x_current_one = x_current_one + 1
            ones_x_sort[img].append(x_current_one)
            ones_y_sort[img].append(y_current_one)
            #print("right")
            
        #down      
        elif find(ones_list[img][0][0], y_current_one + 1, indices_down) != None:
            ones_list[img][0][0][find(ones_list[img][0][0], y_current_one + 1, indices_down)] = -1
            y_current_one = y_current_one + 1
            ones_x_sort[img].append(x_current_one)
            ones_y_sort[img].append(y_current_one) 
            #print("down")
            
        #left   
        elif find(ones_list[img][0][1], x_current_one - 1, indices_left) != None:
            ones_list[img][0][1][find(ones_list[img][0][1], x_current_one - 1, indices_left)] = -1
            x_current_one = x_current_one - 1
            ones_x_sort[img].append(x_current_one)
            ones_y_sort[img].append(y_current_one)
            #print("left")
        
        else: 
            break
        
    for i in range(len(zeros_x_sort[img])):
        u_k_zeros[img].append(complex(zeros_x_sort[img][i], zeros_y_sort[img][i]))
    for i in range(len(ones_x_sort[img])):            
        u_k_ones[img].append(complex(ones_x_sort[img][i], ones_y_sort[img][i]))
    
    #compute the FFT of the u_k for each images
    f_l_zeros[img] = np.fft.fft(u_k_zeros[img]) 
    f_l_ones[img] = np.fft.fft(u_k_ones[img]) 
    
    #les f1, f2, f3 sont ici pour chaque image ! 
    magnitude = np.zeros(3)
    for i in range(1,4):
        magnitude[i-1] = np.sqrt(np.power(f_l_zeros[img][i].real,2) + np.power(f_l_zeros[img][i].imag,2))
        
    featureZeros[img,0] = magnitude[1]/magnitude[0]
    featureZeros[img,1] = magnitude[2]/magnitude[0]
    
    for i in range(1,4):
        magnitude[i-1] = np.sqrt(np.power(f_l_ones[img][i].real,2) + np.power(f_l_ones[img][i].imag,2))
        
    featureOnes[img,0] = magnitude[1]/magnitude[0]
    featureOnes[img,1] = magnitude[2]/magnitude[0]
    
plt.plot(featureZeros[:,1], featureZeros[:,0], 'ro', label = 'zéros')
plt.plot(featureOnes[:,1], featureOnes[:,0], 'bs', label = 'uns')
plt.legend(loc='upper left')
plt.title("Graphe des features")
plt.xlabel("f3/f1")
plt.ylabel("f2/f1")
plt.show()
    ##############################################
    

Il est difficile de séparer clairement les 1 des 0 sur ce graphe. On remarque néanmoins que rien qu'avec 2 composantes de la DFT des contours des formes, la distinction entre les 1 et les 0 se fait déjà ressentir. Nous avons eu du mal à choisir nos features, parce qu'en gardant uniquement f1, et f2, nous avions de meilleurs résultats. Mais suivant le dataset utilisé, l'invariance par changement d'échelle n'est pas à négligé. Pour des raisons de robustesse, nous avons donc choisi de rester avec les features comme sur le plot ci-dessus.

### 1.3 Additional method(s)
Add your implementation and discussion

Nous allons tester de classifier le dataset en utilisant les features suivants: élongation et compacité. Vu l'allure du dataset, uniquement un de ces feature devrait suffir à reconnaitre un 0 ou un 1, mais comme le but de tp est d'extraire 2 features, nous allons utiliser ces 2 derniers. 
Concernant la compacité, nous allons effectuer un premier traitement d'image permettant de remplir les trous (pour rendre les 0 en cercle plein). L'idée est de simplifier l'étape de détection de contour, tout en assurant une bonne compacité pour les 0 contrairement aux 1.
La compacité est de toute manière invariante par translation, rotation et scalling. Pour rendre l'élongation également invariante par translation, rotation, et scalling, l'idée est de considérer uniquement le ratio entre la plus grande et la plus petite valeur propre. 
L'algorithme que nous avons implémenté pour remplir facilement les trous n'est cependant pas invariant par rotation (puisqu'il agit sur un seul axe), toutefois, la classification ne devrait pas trop être affectée (la compacité d'un 1 devrait rester relativement différente de celle d'un 0).

In [ ]:
def fullHoles(img_collection):
    img_collection_copy = np.copy(img_collection)
    nb_of_picture, h, w = img_collection_copy.shape
    for im in img_collection_copy:
        # remplissage en détectant le premier et dernier pixel de chaque colonne
        column = 0
        while (column < w):

            pixel_of_column = 0;
            #search the first and last white pixels
            first = 0;
            while(first < h):
                if(im[first, column] > 0):
                    break
                first = first + 1

            last = (h-1);
            while(last > 0):
                if(im[last, column] > 0):
                    break
                last = last - 1

            if((first == (h-1)) or (last == 0)):
                column = column + 1
                continue
            else:
                for i in range(first, last):
                    im[i,column] = 255
            column = column + 1
        
    return img_collection_copy

In [ ]:
def findContour(img_collection):
    import skimage.filters
    nb_of_picture, h, w = img_collection.shape
    img_collection_copy = np.zeros((nb_of_picture,h,w))
    for i in range(nb_of_picture):
        img_collection_copy[i,:,:] = skimage.filters.roberts(img_collection[i,:,:], mask=None)
        
    return img_collection_copy

In [ ]:
import matplotlib.patches as patches
from skimage import filters
from numpy import linalg as LA
import matplotlib.pyplot as plt

nb_of_picture, h, w = zeros_im.shape
featureZeros = np.zeros((nb_of_picture, 2))
featureOnes = np.zeros((nb_of_picture, 2))

#remplissage des trous
zeros_full = fullHoles(zeros_im)
ones_full = fullHoles(ones_im)
zeros_contour = findContour(zeros_full)
ones_contour = findContour(ones_full)
# calculer les features pour chaque image de 0
for i in range(nb_of_picture):
    # calcul compacité
    aire = len(zeros_full[i,:,:][zeros_full[i,:,:] > 0])
    perimetre = len(zeros_contour[i,:,:][zeros_contour[i,:,:] > 0])
    compacite = perimetre/aire
    featureZeros[i, 0] = compacite
    
    aire = len(ones_full[i,:,:][ones_full[i,:,:] > 0])
    perimetre = len(ones_contour[i,:,:][ones_contour[i,:,:] > 0])
    compacite = perimetre/aire
    featureOnes[i, 0] = compacite
    
    # calcul élongation
    objet = np.where(zeros_im[i,:,:] > 0)
    pixels = np.zeros((2, len(objet[0])))
    pixels[0,:] = objet[0] - np.mean(objet[0])
    pixels[1,:] = objet[1] - np.mean(objet[1])
    cov = np.cov(pixels)
    eigenvalues, eigenvectors = LA.eig(cov)
    elongation = max(eigenvalues)/min(eigenvalues)
    featureZeros[i, 1] = elongation
    
    objet = np.where(ones_im[i,:,:] > 0)
    pixels = np.zeros((2, len(objet[0])))
    pixels[0,:] = objet[0] - np.mean(objet[0])
    pixels[1,:] = objet[1] - np.mean(objet[1])
    cov = np.cov(pixels)
    eigenvalues, eigenvectors = LA.eig(cov)
    elongation = max(eigenvalues)/min(eigenvalues)
    featureOnes[i, 1] = elongation
    
#print(featureZeros)
#print(featureOnes)
plt.plot(featureZeros[:,1], featureZeros[:,0], 'ro', label = 'zéros')
plt.plot(featureOnes[:,1], featureOnes[:,0], 'bs', label = 'uns')
plt.legend(loc='upper left')
plt.title("Graphe des features")
plt.xlabel("élongation (ratio des valeurs propres)")
plt.ylabel("compacité")
plt.show()

Nous constatons que nous pouvons facilement reconnaitre un 0 d'un 1 avec ces features. Il y a une petite confusion entre certains 0 et 1 si l'on considère uniquement l'élongation, contrairement à la compacité. Mais dans l'ensemble, nous pouvons clairement disinguer les 0 des 1 en regardant le plot. 